In [21]:
import os
import csv
from owlready2 import *

Templates for each package are taken from the biosamples ncbi webpage (https://submit.ncbi.nlm.nih.gov/biosample/template/)

'*' indicates mandatory field

'**' indicates at least one attribute in the group required

all other attributes are optional


In [56]:
templates_path = "data/biosamples/templates/"

attributes_dict = {}
required_attributes = set()

for filename in os.listdir(templates_path):
    with open(templates_path + filename, 'r') as f:
        reader = csv.reader(f, delimiter='\t', quotechar='"')
        for row in reader:
            if not row or row[0].startswith("#"):
                continue
            print(row)
            attributes_dict[filename.strip(".tsv")] = [attribute for attribute in row if attribute[0] == "*" and attribute[1] != "*"]

# print(attributes_dict)
# prints the attributes that are common to all biosamples templates (from NCBI packages)
print(set.intersection(*[set(attributes_dict[template]) for template in attributes_dict]))

['*sample_name', 'sample_title', 'bioproject_accession', '*organism', 'strain', 'isolate', '*beta_lactamase_family', '*carbapenemase', '*edta_inhibitor_tested', 'collected_by', 'collection_date', 'geo_loc_name', 'host', 'isolation_source', 'lab_host', 'lat_lon', 'description']
['*sample_name', 'sample_title', 'bioproject_accession', '*organism', '*isolate', '*age', '*biomaterial_provider', '*sex', '*tissue', 'cell_line', 'cell_subtype', 'cell_type', 'culture_collection', 'dev_stage', 'disease', 'disease_stage', 'ethnicity', 'health_state', 'karyotype', 'phenotype', 'population', 'race', 'sample_type', 'treatment', 'description']
['*sample_name', 'sample_title', 'bioproject_accession', '*organism', 'isolate', 'breed', 'host', 'isolation_source', '*collection_date', '*geo_loc_name', '*tissue', 'age', 'altitude', 'biomaterial_provider', 'collected_by', 'depth', 'dev_stage', 'env_broad_scale', 'host_tissue_sampled', 'identified_by', 'lat_lon', 'sex', 'specimen_voucher', 'temp', 'descriptio

In [54]:
onto_path.append("data/ontologies/")
onto = get_ontology("http://purl.obolibrary.org/obo/bto.owl").load()

class_labels = [c.label[0] if len(c.label) > 0 else '' for c in onto.classes()]
# classes = list(c.label[0] for c in onto.classes())
# print(class_labels)
print(len(class_labels))

num_synonyms = 0

for class_label in onto.classes():
    # print(class_label.hasRelatedSynonym)
    num_synonyms += len(class_label.hasRelatedSynonym)


print(num_synonyms)

# direct search for terms in the onotology
# print(onto.search(label="*lea*"))

6569
6208


Things to search for in biosamples XML:

    </BioSample>
    <Organism 
    <Package 
    <Title 
